<h1> Examen Data Science 2019-2020 Partie 2</h1>

<h2> Tous documents aurotisés - 3 heures </h2>

Partie d'examen sur machine, tout documents autorisés. Pour l'anonymisation de votre rendu, veuillez suivre les consignes suivantes : 

- Choisissez un numéro aléatoire à 6 chiffres, le plus aléatoire possible.. Utilisez le même numéro aléatoire pour les 2 parties
- Ecrivez ce numéro ainsi que votre nom et numéro d'étudiant sur la feuille qui circulera en fin d'examen 
- Zippez votre fichier notebook, et nommez l'archive avec votre numéro aléatoire suivi de "_Partie2". Par exempke "127635_Partie2.ipynb
- Envoyez l'archive via la page :  http://stephane.ayache.perso.luminy.univ-amu.fr/examds/upload/upload.php 

 

# Learning to rank


## Introduction 

Le ranking est un problème d'apprentissage différent de la classification ou de la régression, qui consiste à apprendre à ordonner un ensemble d'exemples. 

Un cadre d'application classique est la Recherche d'Information. Un moteur de recherche a pour but de donner un score (de pertinence) à chaque document parmi un ensemble de documents (e.g. les pages du web) pour une requête donnée. Ces scores sont ensuite utilisés pour construire le résultat du moteur de recherche en ordonnant les documents par score de pertinence décroissant. Un moteur de recherche est appris sur un ensemble de requêtes d'apprentissage où un ensemble de documents (éventuellement différent pour chaque requête) est fourni pour chaque requête, avec des scores associés correpesdnant à la pertinence de chaque document pour la requête.

Plus généralement, l'apprentissage d'un modèle (linéaire) de ranking est  formalisé comme suit:
* On considère un ensemble de données d'apprentissage  correspondant à $Q$ requêtes avec, pour chacune des requêtes (par exemple la requête numéro $i$), $N$ exemples associés (par exemple des documents) notés $(x^i_j)_{j=1..N}$ ($x^i_j \in \mathbb{R}^d$ est le $j^{ieme}$ exemple pour la requête $i$) avec leur scores (de pertinence) associés $y^i_j$ (réels ou entiers, une valeur plus grande indiquant une plus grande pertinence). L'ensemble d'apprentissage est noté (en supposant pour simplifier un même nombre de documents, $N$, pour chaque requête mais ce n'est pas obligatoire):
 $$T =\{(x^i_j,y^i_j), i \in [1..Q], j \in [1..N]\}$$ 
* Pour information, mais ce n'est pas important pour la suite, en réalité $x^i_j$ ne correspond pas à un exemple (e.g. un document) mais à un vecteur de caractéristiques joint calculé à partir de cet exemle et de la requête pour laquelle il est considéré.
* Le ranking consiste à apprendre un vecteur de paramètres $w$ à partir de l'ensemble $T$ tel que: $$\forall i \in [1..Q], \forall (j,k) \in [1..N]^2, y^i_j \geq y^i_k \Rightarrow w^T. x^i_j \geq w^T  x^i_k$$
Ce qui compte c'est que les N exemples corrspondant  une même requête soient scorés par le modèle dans l'ordre de leurs pertinences. 









### Montage de votre drive

In [2]:
import os
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [0]:
# Mettez le chemin vers les jeux de données TrainRanking.pickle et TestRanking.pickle 

os.chdir("drive/My Drive/univ/master-2/data-science/exam/")

## Chargement de jeu de données

In [4]:
%matplotlib inline
from os.path import expanduser, join
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
import pickle

with open('TrainRanking.pickle', 'rb') as f:
    X_train, y_train, qid_train = pickle.load(f)

with open('TestRanking.pickle', 'rb') as f:
    X_test, y_test, qid_test = pickle.load( f) 

### Examinez le dataset.
Il est constitué de trois matrices, une matrice de train, les $x^i_j$, un vecteur des pertinences des $x$, les $y^i_j$ qui sont les pertinences des $x^i_j$, et un vecteur des requêtes puisque l'on dispose de multiples $x^i_j$ pour une même requête $i$.


* Combien y-a-t-il d'exemples ?
* Combien y-a-t-il de requêtes?
* Combien y a-t-il de documents par requête en moyenne ? 
* Quelles sont les valeurs min et max de pertinence ?



In [6]:
print(f'xtrain {X_train.shape}')
print(f'ytrain {y_train.shape}')
print(f'qidtrain {qid_train.shape}')

print(f'xtest {X_test.shape}')
print(f'ytest {y_test.shape}')
print(f'qidtest {qid_test.shape}')

min_pertinence, max_pertinence = 0, 0

for x in X_train:
    min_x, max_x = np.min(x), np.max(x)
    if min_x < min_pertinence:
        min_pertinence = min_x
    if max_x > max_pertinence:
        max_pertinence = max_x

print(min_pertinence, max_pertinence)

xtrain (10134, 136)
ytrain (10134,)
qidtrain (10134,)
xtest (24021, 136)
ytest (24021, 1)
qidtest (24021, 1)
-60.3717 65909672.0


In [7]:
10134/136

74.51470588235294

### Résultats donnés pour le train 

* 10134 train
* 136 requêtes
* 10134/136 = 74.51 documents par requête
* -60.3717 min pertinence
* 65909672.0 max pertinence

### Mesure de performance NDCG

* Le NDCG est une mesure dévaluation standard pour les problèmes de ranking

* Vous trouverez une définition du critère NDCG ici : https://en.wikipedia.org/wiki/Discounted_cumulative_gain et une implémentation ci-dessous.


Ces mesures utilisent un vecteur des scores de pertinence, en anglais pertinence = relevance) attribués par la méthode pour chacun des éléments associés à une requête donnée et calculent un score. 

In [0]:
def dcg(relevances, rank=10):
    """Discounted cumulative gain at rank (DCG)"""
    relevances = np.asarray(relevances)[:rank]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances / discounts)
 
def ndcg(relevances, rank=10):
    """Normalized discounted cumulative gain (NDGC)"""
    best_dcg = dcg(sorted(relevances, reverse=True), rank)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, rank) / best_dcg

In [9]:
def mean_ndcg(y_true, y_pred, query_ids, rank=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    # assume query_ids are sorted
    ndcg_scores = []
    previous_qid = query_ids[0]
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
            previous_loc = loc
        previous_qid = qid

    chunk = slice(previous_loc, loc + 1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances, rank=rank))
    return np.mean(ndcg_scores)

mean_ndcg([4, 3, 1, 4, 3], [4, 0, 1, 4, 2], [0, 0, 0, 2, 2], rank=10)

0.9795191506818377

In [0]:
def print_evaluation(model, X, y, qid):
    tic = time()
    y_predicted = model.predict(X)
    prediction_time = time() - tic
    print("Prediction time: {:.3f}s".format(prediction_time))
    print("NDCG@5 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=5)))
    print("NDCG@10 score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=10)))
    print("NDCG score: {:.3f}".format(
    mean_ndcg(y, y_predicted, qid, rank=None)))
    print("R2 score: {:.3f}".format(r2_score(y, y_predicted)))

#### Analyse des mesures de performance


* Que vaut-il mieux, un *NDCG* grand ou petit ?
* Quel est le rôle du paramètre *rank* de la fonction *ndcg* ?
* Le *NDCG* est il invariant par translation et scaling du vecteur de scores ? Justifiez la réponse.
* Que fait la fonction *mean_ndcg* ?


* un NDCG grand (1 score parfait)
* le rank permet de ne considérer que les documents les plus relevants
* 
* la fonction mean_ndcg calcule le ndcg moyen pour un ensemble de prédictions en ne considérant que les `rank` meilleurs documents

## Comparaison de méthodes de ranking

### Sélection de sous ensembles des données d'apprentissage et de test

Afin de pouvoir expérimenter certaines des méthodes ci dessous il faut limiter la taille des données. Ce que font les méthodes suivantes en ne considérant qu'un nombre limité des requêtes en train et en test. 

In [0]:
Nqueries_train = 50
Nqueries_test = 50

In [0]:
def subsample(X, y, qid, size, seed=None):
    rng = np.random.RandomState(seed)
    unique_qid = np.unique(qid)
    qid_mask = rng.permutation(len(unique_qid))[:size]
    subset_mask = np.in1d(qid, unique_qid[qid_mask])
    print(subset_mask)
    a =  y[subset_mask]
    return X[subset_mask,:],a, qid[subset_mask]

In [13]:
X_train_small, y_train_small, qid_train_small = subsample(
    X_train, y_train, qid_train, Nqueries_train, seed=0)
print (X_train.shape, X_train_small.shape, y_train.shape, y_train_small.shape)

[False False False ... False False False]
(10134, 136) (2085, 136) (10134,) (2085,)


In [14]:
X_test_small, y_test_small, qid_test_small = subsample(
    X_test, y_test, qid_test, Nqueries_test, seed=0)
print (X_test.shape, X_test_small.shape)

[False False False ... False False False]
(24021, 136) (5449, 136)


### Une première méthode simple pour le ranking: la régression linéaire

* Vous utiliserez ici une première méthode pour le problème de ranking qui consiste en une méthode simple de régression linéaire visant à prédire la valeur de pertinence $y$ à partir de l'élement $x$, en apprenant sur l'ensemble des données de toutes les requêtes.

* Mettez en oeuvre cette méthode et calculez en la performance.
* Vous évaluerez s'il y a overfitting en calculant les performances en train et en test 

In [15]:
from sklearn.linear_model import LinearRegression

clf_ols = LinearRegression()
clf_ols.fit(X_train, y_train)
print('--- ON TRAIN ---')
print_evaluation(clf_ols, X_train, y_train, qid_train)

print('--- ON TEST ---')
print_evaluation(clf_ols, X_test, y_test, qid_test)

--- ON TRAIN ---
Prediction time: 0.009s
NDCG@5 score: 0.567
NDCG@10 score: 0.592
NDCG score: 0.775
R2 score: 0.200
--- ON TEST ---
Prediction time: 0.008s
NDCG@5 score: 0.419
NDCG@10 score: 0.438
NDCG score: 0.960
R2 score: -2.955


**conclusions**

* il semblerait que l'algorithme fonctione très bien lorsqu'on ne fixe pas de rang.

* si un rang est fixé, alors on est clairement en sur-apprentissage

### Une deuxième méthode basée sur les SVMs : SVMrank

SVMRank est une méthode de ranking inspirée du formalisme des SVMs. SVMRank vise à se ramener à l'apprentissage d'un SVM pour la classification binaire. L'algorithme fonctionne comme suit:

* On construit une nouvelle base d'apprentissage à partir de toutes les paires d'exemples (uniquement les paires d'exemples correspondant à une même requête et à des scores de pertinence différents) en calculant la différence entre les exemples de la paire, c'est à dire que l'on calcule $ \forall i \in Q, \forall j,k \in [1..N]^2, (x^i_j- x^i_k)$, qui constituent les exemples de cette base d'apprentissage. Pour chaque nouvel exemple de cette base (e.g. $(x^i_j- x^i_k)$) la classe à prédire est $1$ si $y^i_j \geq y^i_k$  et elle est égale à $-1$ sinon.  

* On apprend un SVM (on ne considérera que des SVMs linéaires ici) à classifier cette nouvelle base d'apprentissage, ce qui produit un vecteur de paramètres $w$ après apprentissage. A noter, le score de pertinence produit pour une entrée $x^i_j$ par le modèle est donné par $w^T x^i_j$. 

* En inférence, pour une requête donnée on calcule le score de tous les exemples associés à la requête (ce sont les scores prédits de pertinence de ces exemples) et on calcule un NDCG.

1. En vous inspirant par exemple du code ici : https://gist.github.com/coreylynch/4150976/1a2983d3a896f4caba33e1a406a5c48962e606c0
 * Programmez SVMRank en utilisant comme classifieur binaire *sklearn.linear_model.SGDClassifier* 
 * Evaluez SVMRank sur les données et comparez ses performances à la baseline. Vous pourrez utiliser *decision_function* pour calculer les scores de pertinence.



In [0]:
import itertools

import numpy as np

from sklearn.linear_model import SGDClassifier
from sklearn import metrics


def transform_pairwise(X, y):
    """Transforms data into pairs with balanced labels for ranking
    Transforms a n-class ranking problem into a two-class classification
    problem. Subclasses implementing particular strategies for choosing
    pairs should override this method.
    In this method, all pairs are choosen, except for those that have the
    same target value. The output is an array of balanced classes, i.e.
    there are the same number of -1 as +1
    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        The data
    y : array, shape (n_samples,) or (n_samples, 2)
        Target labels. If it's a 2D array, the second column represents
        the grouping of samples, i.e., samples with different groups will
        not be considered.
    Returns
    -------
    X_trans : array, shape (k, n_feaures)
        Data as pairs
    y_trans : array, shape (k,)
        Output class labels, where classes have values {-1, +1}
    """
    X_new = []
    y_new = []
    y = np.asarray(y)
    if y.ndim == 1:
        y = np.c_[y, np.ones(y.shape[0])]
    comb = itertools.combinations(range(X.shape[0]), 2)
    for k, (i, j) in enumerate(comb):
        if y[i, 0] == y[j, 0] or y[i, 1] != y[j, 1]:
            # skip if same target or different group
            continue
        X_new.append(X[i] - X[j])
        y_new.append(np.sign(y[i, 0] - y[j, 0]))
        # output balanced classes
        if y_new[-1] != (-1) ** k:
            y_new[-1] = - y_new[-1]
            X_new[-1] = - X_new[-1]
    return np.asarray(X_new), np.asarray(y_new).ravel()


class RankSVM(SGDClassifier):
    """Performs pairwise ranking with an underlying SGDClassifer model
    Input should be a n-class ranking problem, this object will convert it
    into a two-class classification problem, a setting known as
    `pairwise ranking`.
    Authors: Fabian Pedregosa <fabian@fseoane.net>
             Alexandre Gramfort <alexandre.gramfort@inria.fr>
    https://gist.github.com/2071994
    """

    def fit(self, X, y):
        """
        Fit a pairwise ranking model.
        Parameters
        ----------
        X : array, shape (n_samples, n_features)
        y : array, shape (n_samples,) or (n_samples, 2)
        Returns
        -------
        self
        """
        X_trans, y_trans = transform_pairwise(X, y)
        super(RankSVM, self).fit(X_trans, y_trans)
        return self

    def predict(self, X):
        pred = super(RankSVM, self).predict(X)
        # preds are mapped to {-1,1}
        # FIXME only works in this example!!!
        pred[pred == -1] = 0
        return pred

    def score(self, X, y):
        """
        Because we transformed into a pairwise problem, chance level is at 0.5
        """
        X_trans, y_trans = transform_pairwise(X, y)
        return np.mean(super(RankSVM, self).predict(X_trans) == y_trans)


2. Les scores NDCG obtenus sont-ils comparables avec ceux obtenus avec la régression linéaire ? Est-ce normal ?

In [17]:
clf_ols = LinearRegression()
clf_ols.fit(X_train_small, y_train_small)
print('--- OLS ON TRAIN ---')
print_evaluation(clf_ols, X_train_small, y_train_small, qid_train_small)

print('--- OLS ON TEST ---')
print_evaluation(clf_ols, X_test_small, y_test_small, qid_test_small)

clf_svm = RankSVM()
clf_svm.fit(X_train_small, y_train_small)
print('--- SVM ON TRAIN ---')
print_evaluation(clf_svm, X_train_small, y_train_small, qid_train_small)

print('--- SVM ON TEST ---')
print_evaluation(clf_svm, X_test_small, y_test_small, qid_test_small)

--- OLS ON TRAIN ---
Prediction time: 0.001s
NDCG@5 score: 0.650
NDCG@10 score: 0.687
NDCG score: 0.810
R2 score: 0.339
--- OLS ON TEST ---
Prediction time: 0.002s
NDCG@5 score: 0.399
NDCG@10 score: 0.437
NDCG score: 0.940
R2 score: -0.870
--- SVM ON TRAIN ---
Prediction time: 0.002s
NDCG@5 score: 0.431
NDCG@10 score: 0.475
NDCG score: 0.700
R2 score: -0.028
--- SVM ON TEST ---
Prediction time: 0.009s
NDCG@5 score: 0.311
NDCG@10 score: 0.346
NDCG score: 0.940
R2 score: -0.029


**conclusion**

* SVMRank donne des scores comparables à la régression linéaire bien que légèrement en dessous

3. La corrélation de rang de speramn ou le taux de corrélation de Kendall sont peut-être plus adaptés. Voir les liens  https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient et  https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient. 
 * Qu'en pensez vous ? 
 * kendall tau peut-être utilisé via un import *from scipy import stats* puis *stats.kendalltau*. Donne-t-il des résultats plus satisfaisants ?

In [22]:
from scipy import stats

svm_preds = clf_svm.predict(X_test_small)
svm_tau, svm_p_value = stats.kendalltau(y_test_small, svm_preds)
print(f'kendall correlation for SVM {svm_tau}')

ols_preds = clf_ols.predict(X_test_small)
ols_tau, ols_p_value = stats.kendalltau(y_test_small, ols_preds)
print(f'kendall correlation for OLS {ols_tau}')

kendall correlation for SVM 0.24264534451999034
kendall correlation for OLS 0.27542987225634424


**conclusion**

* la corrélation de Kendall semble intéressante pour des données temporelles ou périodiques puisqu'elle permet de dire que des paires de prédiction sont correctes si elles sont numériquement bien placées $(x_i, y_i) < (x_j, y_j)$ ou $(x_i, y_i) > (x_j, y_j)$

* je n'ai pas le temps de vérifier s'il donne des résultats plus satisfaisants dans ce cas donné, il faudrait pour ça analyser les requêtes et documents associés pour vérifier dans quelle mesure telle ou telle métrique est adaptée : ça n'a pas vraiment de sens d'en parler sans regarder les données